TransE将每个实体和关系都表示成一个向量，并假设对于一个存在在知识图谱中的三元组 $(h, r, t)$, 的向量表示满足：

$$
h + r = t
$$

对于每个正确的三元组的优化目标是：

$$
h + r \approx t
$$

对于一个三元组的评分函数为：

$$
f_r(h, t) = ||h + r - t||_{L_1 / L_2}
$$

TransE的损失函数：

$$
L = \sum_{(h, r, t) \in S} \sum_{(h', r', t') \in S'}
\max (0, f_r(h, t) + \gamma - f_{r'}(h', t'))
$$

其中S是所有正样本的集合，S'是所有负样本的集合，对于一个正样本(h, r, t), 负样本(h', r', t')通过随机替换h或t得到， $\gamma$表示间隔，是一个超参。

In [ ]:
import torch
import torch.nn as nn
import torch.functional as F

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

l1_score = lambda x, y: torch.sum(torch.abs(x - y), dim=-1)
l2_score = lambda x, y: torch.sqrt(torch.sum((x - y) ** 2, dim=-1))

In [ ]:
class TransE():
    def __init__(self, args):
        self.entity_num = args.entity_num
        self.rel_num = args.rel_num
        self.embedding_dimension = args.embedding_dim
        self.entity_embedding = nn.Embedding(self.entity_num, self.embedding_dimension).to(DEVICE)
        self.rel_embedding = nn.Embedding(self.rel_num, self.embedding_dimension).to(DEVICE)
        if args.score_func == 'l1':
            self.score_func = l1_score
        else:
            self.score_func = l2_score

    def init_weights(self):
        nn.init.xavier_normal_(self.entity_embedding.weight)
        nn.init.xavier_normal_(self.rel_embedding.weight)

    def forward(self, head, rel, tail):
        vec_head = self.entity_embedding(head).view(-1, self.embedding_dimension)
        vec_rel = self.rel_embedding(rel).view(-1, self.embedding_dimension)
        vec_tail = self.entity_embedding(tail).view(-1, self.embedding_dimension)

        vec_head = F.normalize(vec_head)
        vec_rel = F.normalize(vec_rel)
        vec_tail = F.normalize(vec_tail)

        return self.score_func(vec_head, vec_rel, vec_tail)